# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [33]:
import pandas as pd
import json
from collections import defaultdict
import re
import datetime, time
import itertools

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [34]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

#### Usamos el metodo **head** para ver como estan estructurados los elementos del JSON

In [35]:
datos.head(10)

,actor,result,verb,_id,object,timestamp
0,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.361000+00:00
1,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'https://w3id.org/xapi/serious...,{'id': 'http://adlnet.gov/expapi/verbs/progres...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.317000+00:00
2,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'extensions': {'fullscreen': True, 'language'...",{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.316000+00:00
3,"{'name': 'gqoj', 'account': {'homePage': 'http...",NaN,{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:35.297000+00:00
4,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'scene': 'menu'}},{'id': 'https://w3id.org/xapi/seriousgames/ver...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.439000+00:00
5,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Estrellas por cate...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.435000+00:00
6,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:42.434000+00:00
7,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': 'Menús: Puedes eleg...,{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.688000+00:00
8,"{'name': 'gqoj', 'account': {'homePage': 'http...","{'score': {'raw': 1}, 'extensions': {'content'...",{'id': 'http://adlnet.gov/expapi/verbs/complet...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:40.685000+00:00
9,"{'name': 'gqoj', 'account': {'homePage': 'http...",{'extensions': {'content': ': '}},{'id': 'http://adlnet.gov/expapi/verbs/initial...,628179b5a6ba41003d3bada2,{'definition': {'type': 'https://w3id.org/xapi...,2022-05-16 10:36:46.017000+00:00


#### Para empezar de forma mas sencilla analizamos el tiempo de juego de un jugador concreto, por ejemplo, de *gqoj*

* Lo primero sería extraer su primer *timestamp*  

In [36]:
inicio = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        inicio = i[5]
        break

In [37]:
inicio

Timestamp('2022-05-16 10:36:35.361000+0000', tz='UTC')

* Ahora que ya tenemos el inicio de la partida vamos a buscar el fin de la partida, para ello nos recorremos todo el JSON y nos quedamos con el último *timestamp* del jugador

In [38]:
fin = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        fin = i[5]

In [39]:
fin

Timestamp('2022-05-23 16:01:55.895000+0000', tz='UTC')

Parece que las trazas no son todas del mismo día, hay que buscar la forma de **filtrar por días**.

#### Ahora que ya sabemos extraer el *timestamp* de un jugador en concreto vamos a intentar recoger los de todos los jugadores

* Guardaremos los datos en un diccionario cuya clave será el **id** del jugador y su valor una pareja con el *timestamp* de inicio y de fin

In [40]:
tiempos = dict()
for i in datos.to_numpy():
    idUser = i[0]["name"]
    timeStamp = i[5]
    
    if(idUser in tiempos):
        timeStampIni = tiempos[idUser][0]
        tiempos[idUser] = (timeStampIni, timeStamp)
    else:
        tiempos[idUser] = (timeStamp, "")

In [41]:
pd.DataFrame(tiempos)

,gqoj,hgfx,gton,uala,qeqyj,fkmtp,hjpbw,bdwyj,eprdu,ftpnb,...,bajbm,eayrh,orjyn,prpej,fmupf,vkfnt,xbmmw,pleui,olugf,epjcx
0,2022-05-16 10:36:35.361000+00:00,2022-05-16 10:48:06.634000+00:00,2022-05-16 22:12:06.760000+00:00,2022-05-17 07:39:39.267000+00:00,2022-05-18 07:10:24.601000+00:00,2022-05-18 07:10:44.894000+00:00,2022-05-18 07:10:59.919000+00:00,2022-05-18 07:11:02.294000+00:00,2022-05-18 07:11:01.092000+00:00,2022-05-18 07:11:18.662000+00:00,...,2022-05-24 09:33:01.510000+00:00,2022-05-24 09:33:02.422000+00:00,2022-05-24 09:33:07.855000+00:00,2022-05-24 09:33:41.060000+00:00,2022-05-24 09:34:20.130000+00:00,2022-05-24 09:34:23.401000+00:00,2022-05-24 09:34:22.295000+00:00,2022-05-24 09:34:32.727000+00:00,2022-05-24 09:36:36.250000+00:00,2022-05-24 12:20:00.205000+00:00
1,2022-05-23 16:01:55.895000+00:00,2022-05-16 10:54:01.016000+00:00,2022-05-16 22:12:21.778000+00:00,2022-05-17 07:42:45.832000+00:00,2022-05-18 10:10:50.954000+00:00,2022-05-18 10:10:45.117000+00:00,2022-05-18 10:10:58.339000+00:00,2022-05-18 10:13:10.064000+00:00,2022-05-18 10:10:53.289000+00:00,2022-05-18 10:10:55.723000+00:00,...,2022-05-24 12:13:54.834000+00:00,2022-05-24 12:12:45.295000+00:00,2022-05-24 12:14:02.923000+00:00,2022-05-24 12:09:03.889000+00:00,2022-05-24 12:11:58.891000+00:00,2022-05-24 12:13:18.012000+00:00,2022-05-24 12:12:04.445000+00:00,2022-05-24 12:12:17.773000+00:00,2022-05-24 12:09:25.385000+00:00,2022-05-24 12:58:14.766000+00:00


* Parece que ya tenemos los datos que queríamos de cada jugador, asumimos que son los correctos porque los tiempos del jugador *gqoj* coinciden con los que hemos sacado antes

* Ahora toca hacer la resta para ver el tiempo de juego de los jugadores 

* Ahora calcularemos el tiempo total de juego de los jugadores

In [54]:
def extraerTiempos(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)
    anomalias = []

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            timestamp = evento["timestamp"]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : "", "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : "", "stars" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
    return data, anomalias   



#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final):
        self.hours = 0
        self.minutes = 0
        self.seconds = 0
        
        fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
        ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
        
        hoursRaw = int((fin - ini)/3600)
        minutesRaw = int((fin - ini)/60)
        secondsRaw = int(fin - ini)
        
        if hoursRaw != 0:
            self.hours = hoursRaw
            minutesRaw = int(((fin - ini)%3600)/60)
        if minutesRaw != 0:
            self.minutes = minutesRaw
            secondsRaw = int((fin - ini)%60)
        self.seconds = secondsRaw
    
    def toString(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        return t

def sumaTiempos(tiempo1, tiempo2):
    nuevaLista = []
    subtiempo1 = [int(s) for s in re.findall(r'-?\d+\.?\d*', tiempo1)]
    subtiempo2 = [int(s) for s in re.findall(r'-?\d+\.?\d*', tiempo2)]

    #Quitamos el numero de estrellas
    subtiempo2.pop()

    #Damos la vuelta para sumar por los segundos
    subtiempo1.reverse()
    subtiempo2.reverse()

    #Sumamos por posicion
    nuevaLista = list(map(sum, itertools.zip_longest(subtiempo1, subtiempo2, fillvalue=0)))

    #Si alguna posicion es 60 o mas, se suma el cociente a la siguiente posicion
    for i in range(len(nuevaLista)):
        if(nuevaLista[i] > 60):
            cociente = nuevaLista[i] // 60
            if(i+2 > len(nuevaLista)):
                nuevaLista.append(cociente)
            else:
                nuevaLista[i+1] = cociente + nuevaLista[i+1]
            resto = nuevaLista[i] % 60
            nuevaLista[i] = resto
  
    #Reconstruimos el string de salida
    if(len(nuevaLista)==1):
        final = str(nuevaLista[0]) + "s"
    elif(len(nuevaLista)==2):
        final = str(nuevaLista[1]) + "m/" + str(nuevaLista[0]) + "s"
    else:
        final = str(nuevaLista[2]) + "h/" + str(nuevaLista[1]) + "m/" + str(nuevaLista[0]) + "s"

    return final


def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"]).toString()
                        if level in tiemposJugados[player]:
                            #Buscamos los ints ya guardados
                            tiempo = [int(s) for s in re.findall(r'-?\d+\.?\d*', str(tiemposJugados[player][level]))]
                            tiemposJugados[player][level].pop()
                            tiemposJugados[player][level].append({"time" : sumaTiempos(timeDifference, str(tiempo)), "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados


JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data, anomalias = extraerTiempos(rawData)
JSONFile.close()

tiempos = tiempoPorNiveles(data, anomalias)
pd.DataFrame(tiempos)

,gqoj,hgfx,uala,qeqyj,fkmtp,bdwyj,eprdu,hjpbw,pbgky,ftpnb,...,eayrh,tcvyz,prpej,bajbm,fmupf,xbmmw,pleui,vkfnt,olugf,epjcx
tutorials_1,"[{'time': '1m/39s', 'stars': 3}]","[{'time': '11s', 'stars': 3}]","[{'time': '40s', 'stars': 3}]","[{'time': '25s', 'stars': 3}]","[{'time': '25s', 'stars': 3}]","[{'time': '57s', 'stars': 3}]","[{'time': '40s', 'stars': 3}]","[{'time': '52s', 'stars': 3}]","[{'time': '26s', 'stars': 3}]","[{'time': '25s', 'stars': 3}]",...,"[{'time': '1m/', 'stars': 3}]","[{'time': '1m/31s', 'stars': 3}]","[{'time': '2m/35s', 'stars': 3}]","[{'time': '1m/17s', 'stars': 3}]","[{'time': '2m/46s', 'stars': 3}]","[{'time': '1m/46s', 'stars': 3}]","[{'time': '1m/34s', 'stars': 3}]","[{'time': '1m/35s', 'stars': 3}]","[{'time': '20s', 'stars': 3}]","[{'time': '1m/40s', 'stars': 3}]"
tutorials_2,"[{'time': '1m/16s', 'stars': 3}]",NaN,"[{'time': '28s', 'stars': 3}]","[{'time': '1m/12s', 'stars': 3}]","[{'time': '39s', 'stars': 3}]","[{'time': '26s', 'stars': 3}]","[{'time': '21s', 'stars': 3}]","[{'time': '51s', 'stars': 3}]","[{'time': '23s', 'stars': 3}]","[{'time': '50s', 'stars': 3}]",...,"[{'time': '26s', 'stars': 3}]","[{'time': '2m/5s', 'stars': 2}]","[{'time': '40s', 'stars': 2}]","[{'time': '2m/4s', 'stars': 2}]","[{'time': '1m/45s', 'stars': 3}]","[{'time': '56s', 'stars': 2}]","[{'time': '30s', 'stars': 3}]","[{'time': '33s', 'stars': 3}]","[{'time': '27s', 'stars': 3}]","[{'time': '2m/18s', 'stars': 3}]"
tutorials_3,"[{'time': '1m/23s', 'stars': 3}]",NaN,"[{'time': '22s', 'stars': 3}]","[{'time': '23s', 'stars': 3}]","[{'time': '22s', 'stars': 3}]","[{'time': '35s', 'stars': 3}]","[{'time': '19s', 'stars': 3}]","[{'time': '29s', 'stars': 3}]","[{'time': '19s', 'stars': 3}]","[{'time': '20s', 'stars': 3}]",...,"[{'time': '19s', 'stars': 3}]","[{'time': '1m/13s', 'stars': 3}]","[{'time': '38s', 'stars': 3}]","[{'time': '42s', 'stars': 3}]","[{'time': '39s', 'stars': 3}]","[{'time': '24s', 'stars': 3}]","[{'time': '31s', 'stars': 3}]","[{'time': '40s', 'stars': 3}]","[{'time': '3m/23s', 'stars': 3}]","[{'time': '30s', 'stars': 3}]"
tutorials_4,"[{'time': '5m/58s', 'stars': 3}]",NaN,NaN,"[{'time': '2m/33s', 'stars': 3}]","[{'time': '2m/25s', 'stars': 3}]","[{'time': '2m/2s', 'stars': 3}]","[{'time': '1m/30s', 'stars': 3}]","[{'time': '3m/7s', 'stars': 3}]","[{'time': '1m/34s', 'stars': 3}]","[{'time': '1m/42s', 'stars': 3}]",...,"[{'time': '2m/31s', 'stars': 3}]","[{'time': '4m/36s', 'stars': 3}]","[{'time': '2m/47s', 'stars': 3}]","[{'time': '3m/25s', 'stars': 3}]","[{'time': '1m/13s', 'stars': 3}]","[{'time': '1m/38s', 'stars': 3}]","[{'time': '1m/24s', 'stars': 3}]","[{'time': '11m/', 'stars': 3}]","[{'time': '7m/54s', 'stars': 3}]","[{'time': '4m/33s', 'stars': 3}]"
tutorials_5,"[{'time': '51s', 'stars': 3}]",NaN,NaN,"[{'time': '24s', 'stars': 3}]","[{'time': '19s', 'stars': 3}]","[{'time': '2m/55s', 'stars': 3}]","[{'time': '19s', 'stars': 3}]","[{'time': '50s', 'stars': 3}]","[{'time': '15s', 'stars': 3}]","[{'time': '39s', 'stars': 3}]",...,"[{'time': '54s', 'stars': 3}]","[{'time': '46s', 'stars': 2}]","[{'time': '27s', 'stars': 3}]","[{'time': '59s', 'stars': 3}]","[{'time': '23s', 'stars': 3}]","[{'time': '18s', 'stars': 3}]","[{'time': '32s', 'stars': 3}]","[{'time': '33s', 'stars': 3}]","[{'time': '22s', 'stars': 3}]","[{'time': '29s', 'stars': 3}]"
tutorials_6,"[{'time': '2m/37s', 'stars': 3}]",NaN,NaN,"[{'time': '1m/32s', 'stars': 2}]","[{'time': '1m/15s', 'stars': 3}]","[{'time': '2m/58s', 'stars': 3}]","[{'time': '1m/6s', 'stars': 3}]","[{'time': '1m/47s', 'stars': 3}]","[{'time': '1m/33s', 'stars': 3}]","[{'time': '1m/10s', 'stars': 2}]",...,"[{'time': '1m/38s', 'stars': 3}]","[{'time': '2m/24s', 'stars': 3}]","[{'time': '1m/30s', 'stars': 3}]","[{'time': '2m/26s', 'stars': 3}]","[{'time': '4m/40s', 'stars': 3}]","[{'time': '1m/22s', 'stars': 3}]","[{'time': '1m/58s', 'stars': 3}]","[{'time': '12m/59s', 'stars': 3}]","[{'time': '18m/58s', 'stars': 3}]","[{'time': '4m/11s', 'stars': 3}]"
tutori